In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [44]:
df=pd.read_csv('final_data.csv')

In [45]:
#Choose relevant columns
#get dummy data
#train test split
#multiple linear regression
#lasso regression
#random forest
#xgboost
#Adaboost
#tune hyperparameters
#test ensembles

In [46]:
#Choose relevant columns
df.columns

Index(['Superficie', 'Rooms', 'Bedrooms', 'Price', 'Location', 'description',
       'Loc_Paris', 'm2', 'nb_bat', 'nb_good', 'nb_bain', 'Cave', 'balc_terr',
       'Chauffage', 'Rénové', 'Parking', 'Chauffage_Central',
       'Chauffage_Individuel', 'Price_int', 'Last_etage'],
      dtype='object')

In [47]:
#Choose relevant columns
df=df[['Rooms', 'Bedrooms','Loc_Paris', 'm2', 'nb_bat', 'nb_good', 'nb_bain', 'Cave', 'balc_terr', 'Rénové', 'Parking', 'Chauffage_Central',
       'Chauffage_Individuel', 'Price_int', 'Last_etage']]

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Rooms                 2557 non-null   object 
 1   Bedrooms              2557 non-null   object 
 2   Loc_Paris             2557 non-null   int64  
 3   m2                    2557 non-null   int64  
 4   nb_bat                2557 non-null   int64  
 5   nb_good               2557 non-null   int64  
 6   nb_bain               2557 non-null   int64  
 7   Cave                  2557 non-null   int64  
 8   balc_terr             2557 non-null   object 
 9   Rénové                2557 non-null   int64  
 10  Parking               2557 non-null   int64  
 11  Chauffage_Central     2557 non-null   int64  
 12  Chauffage_Individuel  2557 non-null   int64  
 13  Price_int             2557 non-null   float64
 14  Last_etage            2557 non-null   int64  
dtypes: float64(1), int64(

In [49]:
#get dummy data
#CHeck the cardinality of our categorical features to know if we wil one hot encode or label encode
df=pd.get_dummies(df)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Loc_Paris                   2557 non-null   int64  
 1   m2                          2557 non-null   int64  
 2   nb_bat                      2557 non-null   int64  
 3   nb_good                     2557 non-null   int64  
 4   nb_bain                     2557 non-null   int64  
 5   Cave                        2557 non-null   int64  
 6   Rénové                      2557 non-null   int64  
 7   Parking                     2557 non-null   int64  
 8   Chauffage_Central           2557 non-null   int64  
 9   Chauffage_Individuel        2557 non-null   int64  
 10  Price_int                   2557 non-null   float64
 11  Last_etage                  2557 non-null   int64  
 12  Rooms_1 p.                  2557 non-null   uint8  
 13  Rooms_14 p.                 2557 

In [58]:
#train test split
from sklearn.model_selection import train_test_split
X=df.drop('Price_int',axis=1)
y=df.Price_int
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [64]:
#random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

rf=RandomForestRegressor()
np.mean(cross_val_score(rf,X_train,y_train,scoring='neg_mean_squared_error',cv=3))

-0.057352743395546545